In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, # GQN
                                                                   labels=next_states_labels))
    eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                     labels=next_states_labels)) # maximize loss
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels)) # minimize loss
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                  action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a')
    targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
    eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
    eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
    aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
#     eloss += tf.reduce_mean((aQs_logits+aQs2_logits)/2) # min
#     aloss2 += -tf.reduce_mean((aQs_logits+aQs2_logits)/2) # max
    eloss += tf.reduce_mean(aQs2_logits) # minimize Q
    aloss2 += -tf.reduce_mean(aQs2_logits) # maxizmie Q
    eloss += tf.reduce_mean(aQs_logits) # minimize Q
    aloss2 += -tf.reduce_mean(aQs_logits) # maxizmie Q
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:15.0000 R:15.0000 rate:0.0300 aloss:0.7038 eloss:2.9847 aloss2:2.3129 exploreP:0.9985
Episode:1 meanR:17.0000 R:19.0000 rate:0.0380 aloss:0.7026 eloss:2.9632 aloss2:2.3474 exploreP:0.9966
Episode:2 meanR:15.0000 R:11.0000 rate:0.0220 aloss:0.7036 eloss:2.9751 aloss2:2.3218 exploreP:0.9956
Episode:3 meanR:19.0000 R:31.0000 rate:0.0620 aloss:0.7000 eloss:2.9460 aloss2:2.3823 exploreP:0.9925
Episode:4 meanR:22.2000 R:35.0000 rate:0.0700 aloss:0.7011 eloss:2.9304 aloss2:2.4151 exploreP:0.9891
Episode:5 meanR:20.0000 R:9.0000 rate:0.0180 aloss:0.6980 eloss:2.9388 aloss2:2.4456 exploreP:0.9882
Episode:6 meanR:21.2857 R:29.0000 rate:0.0580 aloss:0.6993 eloss:2.9440 aloss2:2.4260 exploreP:0.9854
Episode:7 meanR:20.6250 R:16.0000 rate:0.0320 aloss:0.6972 eloss:2.9136 aloss2:2.4688 exploreP:0.9838
Episode:8 meanR:20.7778 R:22.0000 rate:0.0440 aloss:0.6978 eloss:2.9134 aloss2:2.5040 exploreP:0.9817
Episode:9 meanR:20.8000 R:21.0000 rate:0.0420 aloss:0.6994 eloss:2.9306 aloss2:2.47

Episode:80 meanR:29.8148 R:20.0000 rate:0.0400 aloss:0.7468 eloss:3.3612 aloss2:3.0587 exploreP:0.7876
Episode:81 meanR:29.5976 R:12.0000 rate:0.0240 aloss:0.7478 eloss:3.3872 aloss2:3.1268 exploreP:0.7867
Episode:82 meanR:29.6024 R:30.0000 rate:0.0600 aloss:0.7851 eloss:3.4585 aloss2:2.9847 exploreP:0.7843
Episode:83 meanR:29.5238 R:23.0000 rate:0.0460 aloss:0.7610 eloss:3.3716 aloss2:3.0956 exploreP:0.7826
Episode:84 meanR:29.3765 R:17.0000 rate:0.0340 aloss:0.7312 eloss:3.3606 aloss2:3.0535 exploreP:0.7812
Episode:85 meanR:29.6860 R:56.0000 rate:0.1120 aloss:0.7485 eloss:3.3513 aloss2:3.1913 exploreP:0.7769
Episode:86 meanR:29.5517 R:18.0000 rate:0.0360 aloss:0.7473 eloss:3.4408 aloss2:3.0549 exploreP:0.7756
Episode:87 meanR:29.8295 R:54.0000 rate:0.1080 aloss:0.7338 eloss:3.3576 aloss2:3.1752 exploreP:0.7714
Episode:88 meanR:29.8202 R:29.0000 rate:0.0580 aloss:0.7668 eloss:3.3820 aloss2:3.1147 exploreP:0.7692
Episode:89 meanR:29.7222 R:21.0000 rate:0.0420 aloss:0.7120 eloss:3.3796 

Episode:159 meanR:42.5400 R:92.0000 rate:0.1840 aloss:0.6944 eloss:3.0609 aloss2:3.1845 exploreP:0.5582
Episode:160 meanR:43.4100 R:101.0000 rate:0.2020 aloss:0.7074 eloss:3.0745 aloss2:3.2075 exploreP:0.5527
Episode:161 meanR:43.7500 R:57.0000 rate:0.1140 aloss:0.7374 eloss:3.1715 aloss2:3.1739 exploreP:0.5496
Episode:162 meanR:43.5900 R:22.0000 rate:0.0440 aloss:0.7980 eloss:3.1366 aloss2:3.2941 exploreP:0.5484
Episode:163 meanR:43.3100 R:26.0000 rate:0.0520 aloss:0.7687 eloss:3.1328 aloss2:3.3249 exploreP:0.5470
Episode:164 meanR:43.1700 R:21.0000 rate:0.0420 aloss:0.7858 eloss:3.1899 aloss2:3.2625 exploreP:0.5459
Episode:165 meanR:43.2900 R:40.0000 rate:0.0800 aloss:0.8004 eloss:3.1253 aloss2:3.4164 exploreP:0.5437
Episode:166 meanR:43.4400 R:31.0000 rate:0.0620 aloss:0.7733 eloss:3.0634 aloss2:3.4900 exploreP:0.5421
Episode:167 meanR:42.8700 R:29.0000 rate:0.0580 aloss:0.7426 eloss:3.0804 aloss2:3.4541 exploreP:0.5405
Episode:168 meanR:43.1900 R:49.0000 rate:0.0980 aloss:0.7460 el

Episode:238 meanR:43.2300 R:54.0000 rate:0.1080 aloss:0.6953 eloss:2.9834 aloss2:3.2409 exploreP:0.4090
Episode:239 meanR:43.4200 R:60.0000 rate:0.1200 aloss:0.7001 eloss:2.9797 aloss2:3.2619 exploreP:0.4066
Episode:240 meanR:42.6900 R:57.0000 rate:0.1140 aloss:0.6956 eloss:2.9760 aloss2:3.2387 exploreP:0.4043
Episode:241 meanR:43.0800 R:63.0000 rate:0.1260 aloss:0.6906 eloss:2.9679 aloss2:3.2420 exploreP:0.4019
Episode:242 meanR:42.8100 R:48.0000 rate:0.0960 aloss:0.6859 eloss:2.9601 aloss2:3.2665 exploreP:0.4000
Episode:243 meanR:42.6900 R:63.0000 rate:0.1260 aloss:0.6903 eloss:2.9755 aloss2:3.2547 exploreP:0.3975
Episode:244 meanR:43.5100 R:98.0000 rate:0.1960 aloss:0.6858 eloss:2.9991 aloss2:3.2424 exploreP:0.3938
Episode:245 meanR:44.5700 R:133.0000 rate:0.2660 aloss:0.7245 eloss:3.1596 aloss2:3.1261 exploreP:0.3887
Episode:246 meanR:44.5600 R:20.0000 rate:0.0400 aloss:0.7441 eloss:3.2196 aloss2:3.1411 exploreP:0.3879
Episode:247 meanR:44.7400 R:33.0000 rate:0.0660 aloss:0.7568 el

Episode:317 meanR:88.0600 R:129.0000 rate:0.2580 aloss:0.7365 eloss:3.0611 aloss2:3.3390 exploreP:0.1949
Episode:318 meanR:89.0800 R:115.0000 rate:0.2300 aloss:0.7376 eloss:3.0723 aloss2:3.3240 exploreP:0.1928
Episode:319 meanR:90.2100 R:125.0000 rate:0.2500 aloss:0.7366 eloss:3.0612 aloss2:3.3464 exploreP:0.1905
Episode:320 meanR:90.8300 R:108.0000 rate:0.2160 aloss:0.7288 eloss:3.0551 aloss2:3.3580 exploreP:0.1886
Episode:321 meanR:91.4900 R:115.0000 rate:0.2300 aloss:0.7344 eloss:3.0489 aloss2:3.3847 exploreP:0.1865
Episode:322 meanR:92.0600 R:112.0000 rate:0.2240 aloss:0.7254 eloss:3.0501 aloss2:3.3713 exploreP:0.1845
Episode:323 meanR:92.3300 R:135.0000 rate:0.2700 aloss:0.7291 eloss:3.0378 aloss2:3.3618 exploreP:0.1822
Episode:324 meanR:92.9800 R:128.0000 rate:0.2560 aloss:0.7478 eloss:3.0531 aloss2:3.3617 exploreP:0.1800
Episode:325 meanR:94.1400 R:195.0000 rate:0.3900 aloss:0.7429 eloss:3.0387 aloss2:3.3733 exploreP:0.1767
Episode:326 meanR:95.1000 R:145.0000 rate:0.2900 aloss:

...

Episode:5540 meanR:137.3500 R:174.0000 rate:0.3480 aloss:0.3205 eloss:2.6714 aloss2:12.5396 exploreP:0.0100
Episode:5541 meanR:137.7200 R:126.0000 rate:0.2520 aloss:0.3172 eloss:2.8094 aloss2:12.2695 exploreP:0.0100
Episode:5542 meanR:137.5600 R:115.0000 rate:0.2300 aloss:0.3231 eloss:2.8147 aloss2:12.4631 exploreP:0.0100
Episode:5543 meanR:138.1600 R:149.0000 rate:0.2980 aloss:0.3249 eloss:2.6675 aloss2:12.6520 exploreP:0.0100
Episode:5544 meanR:138.8300 R:152.0000 rate:0.3040 aloss:0.3221 eloss:2.8133 aloss2:12.4461 exploreP:0.0100
Episode:5545 meanR:138.4900 R:116.0000 rate:0.2320 aloss:0.3152 eloss:2.5908 aloss2:12.4560 exploreP:0.0100
Episode:5546 meanR:138.3300 R:136.0000 rate:0.2720 aloss:0.3244 eloss:2.9729 aloss2:12.2989 exploreP:0.0100
Episode:5547 meanR:138.2900 R:107.0000 rate:0.2140 aloss:0.3187 eloss:2.8588 aloss2:12.5733 exploreP:0.0100
Episode:5548 meanR:138.6600 R:182.0000 rate:0.3640 aloss:0.3188 eloss:2.9029 aloss2:12.2166 exploreP:0.0100
Episode:5549 meanR:139.0200 

Episode:5616 meanR:144.9900 R:134.0000 rate:0.2680 aloss:0.3011 eloss:3.0739 aloss2:12.4634 exploreP:0.0100
Episode:5617 meanR:144.9200 R:100.0000 rate:0.2000 aloss:0.2958 eloss:3.1387 aloss2:12.2065 exploreP:0.0100
Episode:5618 meanR:144.6100 R:135.0000 rate:0.2700 aloss:0.3072 eloss:2.9123 aloss2:12.9139 exploreP:0.0100
Episode:5619 meanR:144.5300 R:109.0000 rate:0.2180 aloss:0.2906 eloss:2.9947 aloss2:12.3434 exploreP:0.0100
Episode:5620 meanR:144.3400 R:125.0000 rate:0.2500 aloss:0.2859 eloss:2.7924 aloss2:12.5860 exploreP:0.0100
Episode:5621 meanR:144.0200 R:129.0000 rate:0.2580 aloss:0.3040 eloss:2.8471 aloss2:12.7943 exploreP:0.0100
Episode:5622 meanR:143.6800 R:107.0000 rate:0.2140 aloss:0.2897 eloss:3.0345 aloss2:12.1905 exploreP:0.0100
Episode:5623 meanR:144.3100 R:195.0000 rate:0.3900 aloss:0.3049 eloss:2.6895 aloss2:12.9038 exploreP:0.0100
Episode:5624 meanR:143.9200 R:107.0000 rate:0.2140 aloss:0.2939 eloss:2.9740 aloss2:12.4839 exploreP:0.0100
Episode:5625 meanR:144.1500 

Episode:5692 meanR:139.1800 R:145.0000 rate:0.2900 aloss:0.2467 eloss:-0.2762 aloss2:16.0662 exploreP:0.0100
Episode:5693 meanR:138.8800 R:137.0000 rate:0.2740 aloss:0.2497 eloss:-0.1658 aloss2:15.9668 exploreP:0.0100
Episode:5694 meanR:138.0500 R:81.0000 rate:0.1620 aloss:0.2473 eloss:-0.0866 aloss2:15.9358 exploreP:0.0100
Episode:5695 meanR:138.1800 R:150.0000 rate:0.3000 aloss:0.2527 eloss:-0.0746 aloss2:16.0089 exploreP:0.0100
Episode:5696 meanR:138.0100 R:105.0000 rate:0.2100 aloss:0.2495 eloss:-0.1184 aloss2:16.1187 exploreP:0.0100
Episode:5697 meanR:137.8200 R:162.0000 rate:0.3240 aloss:0.2535 eloss:-0.0941 aloss2:16.3788 exploreP:0.0100
Episode:5698 meanR:138.1100 R:182.0000 rate:0.3640 aloss:0.2521 eloss:-0.1492 aloss2:16.2201 exploreP:0.0100
Episode:5699 meanR:138.6500 R:130.0000 rate:0.2600 aloss:0.2509 eloss:-0.1381 aloss2:16.0895 exploreP:0.0100
Episode:5700 meanR:139.3200 R:212.0000 rate:0.4240 aloss:0.2515 eloss:-0.2574 aloss2:16.2121 exploreP:0.0100
Episode:5701 meanR:1

Episode:5768 meanR:145.6300 R:172.0000 rate:0.3440 aloss:0.2692 eloss:-0.3279 aloss2:16.3799 exploreP:0.0100
Episode:5769 meanR:145.7700 R:136.0000 rate:0.2720 aloss:0.2728 eloss:-0.3532 aloss2:16.4438 exploreP:0.0100
Episode:5770 meanR:146.2500 R:181.0000 rate:0.3620 aloss:0.2691 eloss:-0.3458 aloss2:16.2956 exploreP:0.0100
Episode:5771 meanR:145.4200 R:81.0000 rate:0.1620 aloss:0.2753 eloss:-0.2333 aloss2:16.3931 exploreP:0.0100
Episode:5772 meanR:146.1700 R:163.0000 rate:0.3260 aloss:0.2796 eloss:-0.1382 aloss2:16.4814 exploreP:0.0100
Episode:5773 meanR:145.7900 R:86.0000 rate:0.1720 aloss:0.2798 eloss:-0.0086 aloss2:16.2981 exploreP:0.0100
Episode:5774 meanR:144.8600 R:91.0000 rate:0.1820 aloss:0.2695 eloss:-0.3889 aloss2:16.6589 exploreP:0.0100
Episode:5775 meanR:144.2800 R:144.0000 rate:0.2880 aloss:0.2694 eloss:-0.5950 aloss2:16.6049 exploreP:0.0100
Episode:5776 meanR:144.3800 R:135.0000 rate:0.2700 aloss:0.2743 eloss:-0.4435 aloss2:16.5417 exploreP:0.0100
Episode:5777 meanR:144

Episode:5844 meanR:126.8400 R:80.0000 rate:0.1600 aloss:0.2836 eloss:-1.0070 aloss2:17.2310 exploreP:0.0100
Episode:5845 meanR:126.0600 R:100.0000 rate:0.2000 aloss:0.2897 eloss:-1.0624 aloss2:17.3276 exploreP:0.0100
Episode:5846 meanR:125.9800 R:144.0000 rate:0.2880 aloss:0.2865 eloss:-0.9859 aloss2:17.1353 exploreP:0.0100
Episode:5847 meanR:125.1500 R:76.0000 rate:0.1520 aloss:0.2886 eloss:-0.9708 aloss2:17.2816 exploreP:0.0100
Episode:5848 meanR:124.3000 R:68.0000 rate:0.1360 aloss:0.2922 eloss:-0.8193 aloss2:17.3443 exploreP:0.0100
Episode:5849 meanR:124.2000 R:114.0000 rate:0.2280 aloss:0.2895 eloss:-0.9731 aloss2:17.3724 exploreP:0.0100
Episode:5850 meanR:123.8500 R:109.0000 rate:0.2180 aloss:0.2920 eloss:-0.8136 aloss2:17.4563 exploreP:0.0100
Episode:5851 meanR:122.8400 R:58.0000 rate:0.1160 aloss:0.2933 eloss:-1.1837 aloss2:17.7186 exploreP:0.0100
Episode:5852 meanR:122.2700 R:132.0000 rate:0.2640 aloss:0.2839 eloss:-1.1600 aloss2:17.4833 exploreP:0.0100
Episode:5853 meanR:121.

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
